In [ ]:
#Code originally ran on Google Colab using GPU provided by Google

#Importing packages
import statistics
import copy 
import numpy as np
import pandas as pd
import tensorflow as tf
#import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#Connecting to Google Drive
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#Importing data from Google Drive into pandas dataframe while parsing dates as index
df = pd.read_csv("/content/gdrive/My Drive/Bitcoin.csv", parse_dates = ["date"], index_col = "date") #Change dataset name accordingly
df = df.sort_index()

In [ ]:
#removing unnecesarry index column left behind by R
df = df.drop(["Unnamed: 0"], axis=1)

In [ ]:
#Creating a function that will return the large or big dataset
def featureSelector(crypto, size):
  articles = "articles" + crypto
  sentiment = "sentiment" + crypto
  trend = "trend" + crypto
  wiki = "wiki" + crypto
  if size not in [0, 1]:
    print("wrong size specification")
    pass

  elif size == 0: #small dataset
    return df.loc[:,["open", "high", "low", "close", "volume", "marketCap", articles, sentiment, trend, wiki, #changes articles, sentiment, trend and wiki accordingly
           "wti", "brent", "EPU", "vixClose", "target"]]
  else: #big dataset
    return df.loc[:,["open", "high", "low", "close", "volume", "marketCap", articles, sentiment, "articlesBlockchain", "sentimentBlockchain", "articlesCryptocurrency", "sentimentCryptocurrency", #changes articles, sentiment accordingly
           "articlesCryptocurrencies", "sentimentCryptocurrencies", trend, "trendBlockchain", "trendCryptocurrency", "trendCryptocurrencies", wiki, "wikiBlockchain", "wikiCryptocurrency", #changes trend and wiki accordingly
           "wti", "brent", "EPU", "vixClose", "target"]]

# datasetConstructor("Bitcoin"/"Ethereum"/"Ripple", 0/1) where 0 is small dataset and 1 large
# in case of error: reread from csv 
df = featureSelector("Bitcoin", 1)


In [ ]:
print(df)

               open     high      low    close  ...  brent    EPU  vixClose  target
date                                            ...                                
2017-01-01   963.66  1003.08   958.70   998.33  ...  55.05   2351     12.85       1
2017-01-02   998.62  1031.39   996.70  1021.75  ...  55.05  24204     12.85       1
2017-01-03  1021.60  1044.08  1021.60  1043.84  ...  55.05   8985     12.85       1
2017-01-04  1044.40  1159.42  1044.40  1154.73  ...  54.57  10199     11.85       0
2017-01-05  1156.73  1191.10   910.42  1013.38  ...  54.99  13447     11.67       0
...             ...      ...      ...      ...  ...    ...    ...       ...     ...
2020-04-26  7570.14  7700.59  7561.41  7679.87  ...  15.17  85731     33.29       1
2020-04-27  7679.42  7795.60  7679.42  7795.60  ...  15.17  63720     33.29       1
2020-04-28  7796.97  7814.53  7730.81  7807.06  ...  15.60  48139     33.57       1
2020-04-29  7806.71  8871.75  7786.05  8801.04  ...  17.86  43724     31.23 

In [ ]:
#Constructing train and test set. Training set size hardcoded to be 90%
train_size = int(len(df) * 0.9)
test_size = len(df) - train_size
train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]

print(train.shape, test.shape)

(1094, 26) (122, 26)


In [ ]:
#retrieving array names from dataset and removing target label from this array
features = df.columns.drop("target")
#Fit scaler on train set so that model has no information on test data.
scaler = MinMaxScaler()
scaler = scaler.fit(train[features].to_numpy()) #fit data on train data

train.loc[:, features] = scaler.transform(train[features].to_numpy()) #Scaling all features used and replacing in dataframe 
test.loc[:, features] = scaler.transform(test[features].to_numpy()) #Scaling all features used (scaler fit on train data) and replacing in dataframe


#Warning is thrown because data in train and test set are overwritten by scaled data. both train and test data reference to parts of the 'original' pandas df. 

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
print(test)

                open      high       low  ...       EPU  vixClose  target
date                                      ...                            
2019-12-31  0.348612  0.338009  0.352064  ...  0.273001  0.164656       1
2020-01-01  0.343289  0.333807  0.352347  ...  0.481192  0.118169       0
2020-01-02  0.343698  0.331618  0.339192  ...  0.359533  0.118169       1
2020-01-03  0.332034  0.342080  0.338079  ...  0.026883  0.173172       1
2020-01-04  0.351336  0.342790  0.359734  ...  0.550564  0.167140       1
...              ...       ...       ...  ...       ...       ...     ...
2020-04-26  0.363355  0.356970  0.373560  ...  2.221309  0.856991       1
2020-04-27  0.369198  0.361902  0.380038  ...  1.650706  0.856991       1
2020-04-28  0.375484  0.362884  0.382859  ...  1.246792  0.866927       1
2020-04-29  0.376005  0.417760  0.385891  ...  1.132340  0.783889       0
2020-04-30  0.428996  0.447289  0.426905  ...  1.125548  0.173882       0

[122 rows x 26 columns]


In [ ]:
# Making function that will format input data to the right dimension
def datasetCreator(X, y):
  Xs, ys = [], []
  for i in range(len(X)):
    Xs.append(X.iloc[i:i+1].to_numpy())
    ys.append(y.iloc[i])
  return np.array(Xs), np.array(ys).squeeze()

In [ ]:
# Constructing the actual dataset by using the datasetCreator function
# Hardcoding target column name as it is always the same
X_train, y_train = datasetCreator(train.loc[:, features], train.loc[:, "target"])
#X_val, y_val = datasetCreator(val.loc[:, features], val.loc[:, target]) 
X_test, y_test = datasetCreator(test.loc[:, features], test.loc[:, "target"])

In [ ]:
#Making sure the dataframes have equal rows and have the expected dimension
#Small dataset = 14 columns, Big dataset = 25
print(X_train.shape, y_train.shape)

(1094, 1, 25) (1094,)


In [ ]:
def make_model(X_train, y_train, config, i):

  callback = tf.keras.callbacks.EarlyStopping(monitor = "loss", patience = 20)

  #reading the parameters used in the model
  n_units, n_epochs, n_batch, activation, dropout = config 

  model = keras.Sequential()
  model.add(keras.layers.LSTM(units = n_units, input_shape=(X_train.shape[1], X_train.shape[2])
                             #, return_sequences = True     #UNCOMMENT IN ORDER TO TEST 2ND LSTM LAYER!!
                              )) 
  model.add(keras.layers.Dropout(dropout))
 # model.add(keras.layers.LSTM(units = 32))    #UNCOMMENT IN ORDER TO TEST 2ND LSTM LAYER!!
 # model.add(keras.layers.Dropout(dropout))    #UNCOMMENT IN ORDER TO TEST 2ND LSTM LAYER!!
  model.add(keras.layers.Dense(units = 1, activation = activation))
  model.compile(loss= "binary_crossentropy", optimizer= "adam", metrics = ["accuracy"])
  
  #fit model
  history = model.fit(X_train, y_train, epochs = n_epochs, batch_size = n_batch, validation_split = 0.10, shuffle = False, verbose = 0, callbacks=[callback]) #, callbacks=[callback]
  print(i, "accuracy: {}, validation accuracy: {}".format(history.history["accuracy"][-1], history.history["val_accuracy"][-1]))
  savename = "/content/gdrive/My Drive/junk/Model" + str(i) + ".hdf5"
  model.save(savename)
  
def model_configs():
	#defining the parameters used
  n_units = [256]
  n_epochs = [150]
  n_batch = [32]
  activation = ["sigmoid"]
  dropout = [0.15]
	# create configs
  configs = list()
  for i in n_units:
    for j in n_epochs:
      for k in n_batch:
        for l in activation:
          for m in dropout:
              cfg = [i, j, k, l, m]
              configs.append(cfg)
  print("Total configs: {}".format(len(configs)))
  return configs

configList = model_configs()
for config in configList:
  for i in range(30):
    make_model(X_train, y_train, config, i)
    

Total configs: 1
0 accuracy: 0.6209349632263184, validation accuracy: 0.5363636612892151
1 accuracy: 0.6087398529052734, validation accuracy: 0.5545454621315002
2 accuracy: 0.6138211488723755, validation accuracy: 0.5
3 accuracy: 0.6097561120986938, validation accuracy: 0.5090909004211426
4 accuracy: 0.6056910753250122, validation accuracy: 0.5454545617103577
5 accuracy: 0.6158536672592163, validation accuracy: 0.5
6 accuracy: 0.607723593711853, validation accuracy: 0.5272727012634277
7 accuracy: 0.6209349632263184, validation accuracy: 0.5272727012634277
8 accuracy: 0.6158536672592163, validation accuracy: 0.5272727012634277
9 accuracy: 0.6209349632263184, validation accuracy: 0.5454545617103577
10 accuracy: 0.6168699264526367, validation accuracy: 0.5363636612892151
11 accuracy: 0.6189024448394775, validation accuracy: 0.5363636612892151
12 accuracy: 0.6219512224197388, validation accuracy: 0.5181818008422852
13 accuracy: 0.6260162591934204, validation accuracy: 0.5545454621315002
14

In [ ]:
ethereum_small = keras.models.load_model("/content/gdrive/My Drive/junk/Model7.hdf5")
test_loss_score, test_acc_score = ethereum_small.evaluate(X_test, y_test)
test_loss_score, test_acc_score = ethereum_small.evaluate(X_train, y_train)

35/35 [==============================] - 0s 3ms/step - loss: 0.6635 - accuracy: 0.6115


In [ ]:
test_loss_score, test_acc_score = ethereum_small.evaluate(X_test, y_test)
test_loss_score, test_acc_score = ethereum_small.evaluate(X_train, y_train)

35/35 [==============================] - 0s 3ms/step - loss: 0.6635 - accuracy: 0.6115


In [ ]:
predictions = ripple_small.predict_classes(X_test)



NameError: ignored

In [ ]:
print(y_test)
print(predictions.flatten())

In [ ]:
X_test
X_test.shape